In [15]:
import pandas as pd
from fuzzywuzzy import fuzz
from dateutil.parser import parse

# Load the two CSV files into dataframes
df1 = pd.read_csv('ExampleC3.csv')
df2 = pd.read_csv('ExampleC3_AGOL.csv')

# Find the best match between the two rows
for index1, row1 in df1.iterrows():
    best_match_score = 0
    best_match_row = None

    # Find the best match with any row in df2
    for index2, row2 in df2.iterrows():
        score = fuzz.ratio(str(row1), str(row2))
        if score > best_match_score:
            best_match_score = score
            best_match_row = row2

    # Print out the contents of the best match
    if best_match_score > 0:
        date1 = parse(row1['Final C3 Date'])
        date2 = parse(best_match_row['Final C3 Construction'])
        if date1 == date2:
            print('Best match for row {} in file1.csv:'.format(index1))
            print(row1)
            print('Best match in file2.csv:')
            print(best_match_row)
            print('Values of Final C3 Date and Final C3 Construction match.')
        else:
            print('Best match for row {} in file1.csv:'.format(index1))
            print(row1)
            print('Best match in file2.csv:')
            print(best_match_row)
            print('Values of Final C3 Date and Final C3 Construction do not match.')
    else:
        print('No match found for row {} in file1.csv.'.format(index1))

Best match for row 0 in file1.csv:
Project Name       Big One
Final C3 Date    30-Jan-08
Name: 0, dtype: object
Best match in file2.csv:
Project Description           Big One LLC
Final C3 Construction    30, January 2008
Name: 0, dtype: object
Values of Final C3 Date and Final C3 Construction match.
Best match for row 1 in file1.csv:
Project Name     Small guy / ross
Final C3 Date           24-Feb-18
Name: 1, dtype: object
Best match in file2.csv:
Project Description                Ross
Final C3 Construction    24, march 1999
Name: 1, dtype: object
Values of Final C3 Date and Final C3 Construction do not match.
Best match for row 2 in file1.csv:
Project Name      1874 budda hand
Final C3 Date    17, January 1999
Name: 2, dtype: object
Best match in file2.csv:
Project Description      1874 budda hand St
Final C3 Construction      17, January 1999
Name: 2, dtype: object
Values of Final C3 Date and Final C3 Construction match.
